# Deep Learning Applications

This lesson assumes students are 
- Strong on Data Science fundamentals
- Familiar with Deep Learning theory
- Comfortable with Sci-Kit Learn

At the end of this lesson you should be able to
- Use Keras to build models that tackle many common applications
- Modify existing "cookie-cutter" models to tailor them for specific tasks
- Interact with Keras and Tensorflow where needed

We can't cover everything so keep these resources handy and refer back to them when you need more details
- [Keras documentation](https://keras.io/)
- [Tensorflow documentation](https://www.tensorflow.org/api_docs/)
- [Official Keras examples](https://github.com/keras-team/keras/tree/master/examples) dozens of examples. just about everything you could need
- [Deep Learning Book](http://www.deeplearningbook.org/) Free and just about the best coverage of the theory behind Deep Learning around

In [ ]:
%pylab inline
import pandas as pd 
import keras

IMSIZE = [224,224]
BATCH = 32

# Deep Learning approaches

First, recall from previous lessons the essential parts of any neural network

- A chain of differentiable functions that
  - Includes trainable parameters.
  - Could, with the right parameters, perform the operation you're looking for.
  - Includes [non-linearities](https://stackoverflow.com/questions/9782071/why-must-a-nonlinear-activation-function-be-used-in-a-backpropagation-neural-net).
- A [loss function](http://ml-cheatsheet.readthedocs.io/en/latest/loss_functions.html) that puts a number on "does the network do what I want it to?".
  - Regression (guess the right number): Mean Squared Error, Mean Average Error
  - Categorization (guess the right class): Categorical Cross-Entropy
  - Multi-Label classification (guess which labels apply): Binary Cross-Entropy

Given these two pieces, we use backpropogation to optimize our trainable function to produce better and better output at each step. 


We will learn two methods for accomplishing this:

## 1 The Tensorflow method

Here, we will build the "function" that our model computes piece-by-piece. We'll create every variable, define all the operations that connect them, and "manually" feed data into the function each time.

Here we'll accomplish two things. First, we'll take some of the mystery out of neural nets. This is meant to be a short learning experience. If you wish to include neural nets in your project **it is not recommended to start with this method**. In general, this is an approach you may wish to revisit if, after using the second method, you wish to have more direct control over how your model is built.

## 2 The Keras method 

Here, we use high-level building blocks to quickly compose complex-yet-powerful networks. Keras gives you access to some of the most advanced, difficult to implement architectures without requiring you to implement them by hand. **This is where most neural net projects should start**.

Again, just for complete clarity, these methods are taught in the current order so that, when we arrive the Keras approach, we understand what Keras is doing under the hood. 

## Data prep

We will focus on image recognition, a task Deep Learning models have excelled in in recent years. We will use data from the [Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats/data) Kaggle competition. Follow the link above and download the `train.zip` file under data. Unzip the file and note the location below. 

First we will pull out 1,000 images each for our train, validation, and test sets. This is necessary to execute these models on most laptop computers. If you have access to a more powerful computer or are happy to wait, feel free to adjust the size of the sets.

**TODO:** In the below cell, set the variable dvc_path to the location where you downloaded the dataset. Then run the following cells to create the train, validation, and test set and to visualize a sample of the images. 

In [ ]:
dvc_path = "/home/zephyrie/AI_501"
split_sizes = {"train": 1000, "validation":1000, "test":1000}

In [ ]:
import glob, shutil, os

In [ ]:
labeled_files = set(glob.glob(dvc_path+"/train/**.jpg", recursive=True))
cat_files = {f for f in labeled_files if "cat" in f}
dog_files = labeled_files - cat_files

print(f"found {len(labeled_files)} labeled files, {len(cat_files)} of cats and {len(dog_files)} of dogs")

shutil.rmtree(dvc_path+"/split", ignore_errors=True)

for split in split_sizes:
    print(f"moving {split_sizes[split]} files for {split}")
    n = split_sizes[split]//2
    os.makedirs(f"{dvc_path}/split/{split}/cat", exist_ok=True)
    os.makedirs(f"{dvc_path}/split/{split}/dog", exist_ok=True)
    
    new_cats = set(np.random.choice(list(cat_files), size=n, replace=False))
    for new_cat_file in new_cats:
        _, fname = new_cat_file.split("train/")
        shutil.copy(new_cat_file, f"{dvc_path}/split/{split}/cat/{fname}")
    cat_files = cat_files - new_cats
        
    new_dogs = set(np.random.choice(list(dog_files), size=n, replace=False))
    for new_dog_file in new_dogs:
        _, fname = new_dog_file.split("train/")
        shutil.copy(new_dog_file, f"{dvc_path}/split/{split}/dog/{fname}")
    dog_files = dog_files - new_dogs

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
def plot_images(x, y, **kwargs):
    n_pix = int(np.sqrt(np.prod(x.shape[1:3]))) #assumes images are square
    im_indices = np.random.choice(x.shape[0], 36, replace=False)
    fig, axes = subplots(nrows=6,ncols=6, figsize=(10,10), sharex=True, sharey=True, frameon=False)
    for i,ax in enumerate(axes.flat):
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        curr_i = im_indices[i]
        ax.imshow(x[curr_i].reshape(n_pix,n_pix, 3), aspect="auto", **kwargs)
        if y[curr_i]==0:
            ax.text(10,20,'cat', fontdict={"backgroundcolor": "gray","color": "white" })
        else:
            ax.text(10,20,'dog', fontdict={"backgroundcolor": "gray","color": "white" })
#         ax.set_title(title)
        ax.axis('off')
    plt.tight_layout(h_pad=0, w_pad=0)

In [ ]:
plot_gen = datagen.flow_from_directory(
    directory=dvc_path+"/split/train",
    target_size=IMSIZE,
    batch_size=36,
    class_mode='binary'
)

x,y = next(plot_gen)

plot_images(x, y, interpolation="spline16")

# The Tensorflow method: Deep Learning the hard(er) way

In [ ]:
import tensorflow as tf
from sklearn import datasets, model_selection, preprocessing
tf.__version__

In [ ]:
TF_IMSIZE = [50,50]

datagen = ImageDataGenerator(rescale=1/255, samplewise_center=True, samplewise_std_normalization=True)

train_gen = datagen.flow_from_directory(
    directory=dvc_path+"/split/train",
    target_size=TF_IMSIZE,
    batch_size=BATCH,
)

val_gen = datagen.flow_from_directory(
    directory=dvc_path+"/split/validation",
    target_size=TF_IMSIZE,
    batch_size=1000,
)

In [ ]:
# Network Parameters
n_hidden_1 = 64 # 1st layer number of neurons
n_hidden_2 = 32 # 2nd layer number of neurons
num_input = np.prod(TF_IMSIZE)*3 #
num_classes = 2 # 

# Placeholders are not trainable (this is our input and output)
X = tf.placeholder("float", [None] + TF_IMSIZE + [3])
X_flat = tf.contrib.layers.flatten(X)
Y = tf.placeholder("float", [None, num_classes])

Variables **are** trainable. Below, we define all the varaibles we'll use.

Pay attention here to how the dot product affects dimensions. I'll use the following notation to indicate that $A$ is an array with 100 elements along the first dimension and 50 along the second. In other words `A.shape == (100, 50)`.

$ A_{[100 \times 50]} \cdot B = C $

We need to line up the dimensions of our variables such that the last dimension of $A$ matches the first dimension of $B$. Also, keep in mind that $C$, the output is going to have a shape based on the first dimension of $A$ and the last dimension of $B$.

$ A_{[100 \times 50]} \cdot B_{[50 \times 25]} = C_{[100 \times 25]} $

In [ ]:
# placing them in a dictionary is helpful for keeping organized
# but these are just python variables.
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
# Now we define the operations we'll use to construct
# the output from our inputs and trainable parameters

# First hidden fully connected layer
layer_1 = tf.matmul(X_flat, weights['h1']) + biases['b1']

# Second hidden fully connected layer
layer_2 = tf.matmul(layer_1, weights['h2']) + biases['b2']

# Output fully connected layer with a neuron for each class
logits = tf.matmul(layer_2, weights['out']) + biases['out']

# Is something missing here? What?

In [ ]:
learning_rate = 0.001

# Define the loss and optimizer
# recall that cross-entropy loss is what we use for most categorization problems
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
loss_op = tf.reduce_mean(loss)

# The optimizer uses gradient descent and the backprop algorithm
# Most of these are just variations on Stochastic Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Parameters
num_steps = 350 # roughly 10 epochs
display_step = int(num_steps//20)

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    loss = []
    acc = []

    for step in range(1, num_steps+1):
        batch_x, batch_y = next(train_gen)
        # Run optimization op (backprop)
        _, train_loss, train_acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
        loss.append(train_loss)
        acc.append(train_acc)
        if step % display_step == 0:
            
            x_val, y_val = next(val_gen)
            val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: x_val,
                                                                 Y: y_val})
            print(f"Step {step}, Train: Loss={np.mean(loss):.4f}, Acc={np.mean(acc):.2%}"
                  f"| Val: Loss={val_loss:.4f}, Acc={val_acc:.2%}")
            loss = []
            acc = []

    print("Optimization Finished!")

# **Exercise 1**: 

Explain what is happening here. Specifically, is anything about this network and its performance not ideal? Why?


# **Problem 2:**

Using the above code as a starting point, try to address the problems mentioned above by keeping the two hidden layer structure but adding a non-linearity and re-training the network.

# Keras implementation

Keras is a framework for building tensorflow code. It automates many tensorflow tasks that are often repeated and allows data scientists to rapidly build and modify complex networks.

The tensorflow model we just built is re-implemented below in tensorflow code. Notice how few lines of code we need. In particular, the messy system for running batches of data through a tensorflow session is very clean here.

In [ ]:
TF_IMSIZE = [50,50]

train_gen = datagen.flow_from_directory(
    directory=dvc_path+"/split/train",
    target_size=TF_IMSIZE,
    batch_size=BATCH,
    class_mode='binary'
)

val_gen = datagen.flow_from_directory(
    directory=dvc_path+"/split/validation",
    target_size=TF_IMSIZE,
    batch_size=BATCH,
    class_mode='binary'
)

In [ ]:
# Create a Keras Sequential model
# We do this by passing a list of layers to the Sequential model

model = keras.Sequential([
    keras.layers.InputLayer(input_shape=TF_IMSIZE+[3]),
    keras.layers.Flatten(),
    keras.layers.Dense(20),
    keras.layers.Dense(10),
    keras.layers.Dense(1, activation="sigmoid")
])

model.summary() #summary provides an at-a-glance look at the model we've built

In [ ]:
# Compile the network
model.compile(
    loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

steps = 1000 / BATCH
callbacks = [
    keras.callbacks.ReduceLROnPlateau(
        factor=.5, patience=1, verbose=1, min_lr=1e-8),
    keras.callbacks.EarlyStopping(patience=5, verbose=1),
]

model.fit_generator(
    generator=train_gen,
    steps_per_epoch=steps,
    validation_data=val_gen,
    validation_steps=steps,
    callbacks=callbacks,
    epochs=100)

# ** Exercise 3:**

Extend the above model as we did before by adding nonlinear activation to the existing layers. 

# **Exercise 4:** 

Recall that on image tasks, convolutional networks often greatly outperform feedforward networks like those we've created so far. 

Build a new keras model that uses alternating `Conv2D` layers and `MaxPool2D` (or `AvgPool2D`) layers. Feel free to experiment with adding other layers. Batch normalization often helps deeper networks train and dropout or gaussian noise often helps with overfitting.

# Transfer Learning

Unfortunately, for our specific task of distinguishing pictures of cats from pictures of dogs, there aren't many pre-trained models that we can take advantage of. However, many fantastic models have been pre-trained on the popular imagenet dataset which includes hundreds of thousands of images across hundreds of categories. 

Our last step will be to take a model trained for imagenet and transfer its knowledge to our task. That way, our model doesn't have to learn everything from scratch. 

# **Exercise 5:** 

Load mobilenet from `keras.applications.mobilenet`. Initialize it with 

```
mnet_base = MobileNet(
    input_shape=(128,128,3),
    weights='imagenet',
    alpha=.25,
    pooling="avg",
    include_top=False)
```

Set all the layers in the model to not be trainable (hint: you can access the layers with `model.layers` and set to not trainable with `layer.trainable = False`). Now, build a new sequential model that uses `mnet_base` as a layer. Run this on our same dataset and compare your results. 

mobilenet has pre-trained weights for alphas {0.25, .5, .75, 1} and for image sizes {128, 160, 192,192, 224}. Feel free to experiment with different values.